In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/ColabNotebooks/BA/Experiment/vgg-face-pytorch
!pip install -r requirements.txt

/content/gdrive/MyDrive/ColabNotebooks/BA/Experiment/vgg-face-pytorch
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 27.7 MB/s 
     |████████████████████████████████| 592 kB 71.1 MB/s 
     |████████████████████████████████| 2.3 MB 54.5 MB/s 
     |████████████████████████████████| 3.1 MB 55.5 MB/s 
     |████████████████████████████████| 1.0 MB 46.4 MB/s 
     |████████████████████████████████| 3.5 MB 76.1 MB/s 
     |████████████████████████████████| 34.6 MB 1.2 MB/s 
  Created wheel for torchfile: filename=torchfile-0.1.0-py3-none-any.whl size=5709 sha256=eac00ff556652abe5145afebebf3cd183a48db4000176650dfec71099a486456
  Stored in directory: /root/.cache/pip/wheels/ac/5c/3a/a80e1c65880945c71fd833408cd1e9a8cb7e2f8f37620bb75b
Successfully built torchfile
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.34.0
    Uninstalling llvmlite-0.34.0:
      Successfully

### Dodging

In [ ]:
%cd /content/gdrive/MyDrive/ColabNotebooks/BA/Experiment/vgg-face-pytorch

import torch
import torch.nn as nn
import numpy as np
import cv2
import sys
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torchvision import transforms
from models.utils import preprocess_data
from models.vgg_face import get_pretrained_model, get_prediction
from image_utils import save_image, show_image, compare_images
from art.attacks.evasion import ProjectedGradientDescent # will choose the correct version (e.g. pytorch) based on the classifier (e.g. PyTorchClassifier) used
from art.estimators.classification import PyTorchClassifier
import torch.optim as optim
import logging
from google.colab.patches import cv2_imshow

np.set_printoptions(threshold=sys.maxsize)


logger = logging.getLogger("pgd_attack_dodging-logger")

logging.basicConfig(
    level=logging.INFO,
    filename="logs/pgd_attack_dodging.log",
    filemode="w",
    format="%(asctime)s %(name)s %(levelname)s: %(message)s"
)


names = [line.rstrip('\n') for line in open('data/names.txt')]
 
BATCH_SIZE = 30
NUM_PASSES = 5
ITERA = [1, 2, 3, 5, 7, 10, 15, 20, 30]
ALPHA = 20

model = get_pretrained_model()

model.eval()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)


classifier = PyTorchClassifier(
    model=model,
    loss=criterion,
    optimizer=optimizer,
    nb_classes=2622,
    input_shape=(1, 3, 224, 224),
    device_type="gpu"
)


glass = cv2.imread('data/glasses/silhouette.png')
glasses = glass.transpose(2, 0, 1).astype("float32")


for i in range(len(ITERA)):

  logger.info(f"Num of attack iterations: {ITERA[i]}")
  
  dataloaders, dataset_sizes = preprocess_data(BATCH_SIZE)
  dataiter = iter(dataloaders)

  pgd_attack = ProjectedGradientDescent(estimator=classifier, eps=np.inf, eps_step=ALPHA, norm=np.inf, max_iter=ITERA[i])

  for j in range(NUM_PASSES):

    logger.info(f"Run {j + 1}/{NUM_PASSES}")

    data = dataiter.next()

    images, labels = data

    images = images[:,[2,1,0],:,:].numpy()
    labels = labels.numpy()

    x_adv = pgd_attack.generate(x=images, y=labels, batch_size=BATCH_SIZE, mask=glasses)

    values, indices = get_prediction(torch.from_numpy(x_adv).to(device), model)

    predictions = [(f"id: {index}", f"name: {names[index]}", f"confidence: {value}") for index, value in zip(indices.tolist(), values.tolist())]
    ground_truth = [(f"id: {label}", f"name: {names[label]}") for label in labels.tolist()]  

    logger.info(f"Predictions: {predictions}\n\n"\
                f"Labels: {ground_truth}\n\n" \
                f"Num. predictions: {labels.size}\n" \
                f"Num. correct predictions: {(indices == torch.from_numpy(labels).to(device)).sum().item()}\n\n")
    
  save_image(f"pgd_attack/dodging/iters{ITERA[i]}-{BATCH_SIZE}batch-", torch.from_numpy(x_adv))
 

### Impersonation

In [ ]:
%cd /content/gdrive/MyDrive/ColabNotebooks/BA/Experiment/vgg-face-pytorch

import torch
import torch.nn as nn
import numpy as np
import cv2
import sys
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torchvision import transforms
from models.utils import preprocess_data
from models.vgg_face import get_pretrained_model, get_prediction
from image_utils import save_image, show_image, compare_images
from art.attacks.evasion import ProjectedGradientDescent # will choose the correct version (e.g. pytorch) based on the classifier (e.g. PyTorchClassifier) used
from art.estimators.classification import PyTorchClassifier
import torch.optim as optim
import logging
from google.colab.patches import cv2_imshow
import random

np.set_printoptions(threshold=sys.maxsize)


logger = logging.getLogger("pgd_attack_impersonation-logger")

logging.basicConfig(
    level=logging.INFO,
    filename="logs/pgd_attack_impersonation.log",
    filemode="w",
    format="%(asctime)s %(name)s %(levelname)s: %(message)s"
)


names = [line.rstrip('\n') for line in open('data/names.txt')]
 
BATCH_SIZE = 30
NUM_PASSES = 5
ITERA = 30
ALPHA = 20
IMPERSONATION_ATTEMPTS = 3

model = get_pretrained_model()

model.eval()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)


classifier = PyTorchClassifier(
    model=model,
    loss=criterion,
    optimizer=optimizer,
    nb_classes=2622,
    input_shape=(1, 3, 224, 224),
    device_type="gpu"
)


glass = cv2.imread('data/glasses/silhouette.png')
glasses = glass.transpose(2, 0, 1).astype("float32")

pgd_attack = ProjectedGradientDescent(estimator=classifier, eps=np.inf, eps_step=ALPHA, norm=np.inf, max_iter=ITERA, targeted=True)

for i in range(IMPERSONATION_ATTEMPTS):

  target = random.randint(0, 2621)
  target_tensor = torch.full((BATCH_SIZE,), target).numpy()

  logger.info(f"Attempt: {i + 1} - Target: (id: {target}, name: {names[target]})")
  logger.info(f"Num of attack iterations: {ITERA}")
  
  dataloaders, dataset_sizes = preprocess_data(BATCH_SIZE)
  dataiter = iter(dataloaders)

  for j in range(NUM_PASSES):

    logger.info(f"Run {j + 1}/{NUM_PASSES}")

    data = dataiter.next()

    images, labels = data

    images = images[:,[2,1,0],:,:].numpy()
    labels = labels.numpy()

    x_adv = pgd_attack.generate(x=images, y=target_tensor, batch_size=BATCH_SIZE, mask=glasses)

    values, indices = get_prediction(torch.from_numpy(x_adv).to(device), model)

    predictions = [(f"id: {index}", f"name: {names[index]}", f"confidence: {value}") for index, value in zip(indices.tolist(), values.tolist())]
    ground_truth = [(f"id: {label}", f"name: {names[label]}") for label in labels.tolist()] 

    num_total_predictions = labels.size
    num_correct_predictions = (indices == torch.from_numpy(labels).to(device)).sum().item()
    num_successful_impersonation = (indices == torch.from_numpy(target_tensor).to(device)).sum().item()
    num_dodging = num_total_predictions - num_correct_predictions - num_successful_impersonation 
    
    logger.info(f"\n\nPredictions: {predictions}\n\n"\
                f"Labels: {ground_truth}\n\n" \
                f"Num. total predictions: {num_total_predictions}\n" \
                f"Num. correct predictions (indices == labels): {num_correct_predictions}\n" \
                f"Num. successful IMPERSONATION predictions (indices == targets): {num_successful_impersonation}\n" \
                f"Num. successful DODGING predictions (indices != labels && indices != targets): {num_dodging}\n\n")
  
  save_image(f"pgd_attack/impersonation/attempt{i}-iters{ITERA}-", torch.from_numpy(x_adv))